# chess.com

In [100]:
import concurrent.futures
import pandas
import requests
from tqdm.notebook import tqdm

## Utils

In [101]:
def chunks(list, n):
    # looping till length l
    for i in range(0, len(list), n):
        yield list[i:i + n]

## Declare API

In [102]:
MAIN_API = "https://api.chess.com/pub"
PROXY_API = "https://homapis.vercel.app/api/chess.com"

## Get Titled Players

In [103]:
title_abbreviations = ["GM", "WGM", "IM", "WIM", "FM", "WFM", "NM", "WNM", "CM", "WCM"]

title_players = []

for title_abbreviation in tqdm(title_abbreviations):
    url = f"{MAIN_API}/titled/{title_abbreviation}"
    response = requests.get(url, timeout=10)
    data = response.json()
    players = data["players"]
    for player in players:
        title_players.append({
            "username": player,
            "title_abbreviation": title_abbreviation
        })

title_players_data_frame = pandas.DataFrame.from_dict(title_players)
title_players_data_frame.to_csv(r'data/csv/title_players.csv', index = False, header = True)

  0%|          | 0/10 [00:00<?, ?it/s]

### Chunk Players

In [104]:
chunks_title_players = list(chunks(title_players, 20))

# Get Titled Profiles

### Get Profile

In [105]:
def get_profile(username):
    profile_url = f"{PROXY_API}/player/{username}"
    try:
        response = requests.get(profile_url, timeout=10)
        if response.status_code == 200:
            profile = response.json()
            return profile
        else:
            return {
                "username": username
            }
    except:
        return {
            "username": username
        }

### Get Profiles

In [106]:
title_profiles = []

for chunk_title_players in tqdm(chunks_title_players):
    usernames = list(map(lambda title_player: title_player["username"], chunk_title_players))
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for username in usernames:
            futures.append(executor.submit(get_profile, username=username))
        for future in concurrent.futures.as_completed(futures):
            profile = future.result()
            title_profiles.append(profile)
    title_profiles_data_frame = pandas.DataFrame.from_dict(title_profiles)
    title_profiles_data_frame.to_csv(r'data/csv/title_profiles.csv', index = False, header = True)

  0%|          | 0/558 [00:00<?, ?it/s]

## Get Title Stats

### Get Player's Stats

In [129]:
def get_stats(username):
    stats_url = f"{PROXY_API}/player/{username}/stats"
    try:
        response = requests.get(stats_url, timeout=10)
        if response.status_code == 200:
            stats = response.json()
            fide = stats.get("fide", 0)
            # Rapid
            rapid_rating_best = stats.get("chess_rapid", {}).get("best", {}).get("rating", 0)
            rapid_rating_last = stats.get("chess_rapid", {}).get("last", {}).get("rating", 0)
            rapid_rating_rd = stats.get("chess_rapid", {}).get("last", {}).get("rd", 0)
            rapid_record_win = stats.get("chess_rapid", {}).get("record", {}).get("win", 0)
            rapid_record_draw = stats.get("chess_rapid", {}).get("record", {}).get("draw", 0)
            rapid_record_loss = stats.get("chess_rapid", {}).get("record", {}).get("loss", 0)
            # Blitz
            blitz_rating_best = stats.get("chess_blitz", {}).get("best", {}).get("rating", 0)
            blitz_rating_last = stats.get("chess_blitz", {}).get("last", {}).get("rating", 0)
            blitz_rating_rd = stats.get("chess_blitz", {}).get("last", {}).get("rd", 0)
            blitz_record_win = stats.get("chess_blitz", {}).get("record", {}).get("win", 0)
            blitz_record_draw = stats.get("chess_blitz", {}).get("record", {}).get("draw", 0)
            blitz_record_loss = stats.get("chess_blitz", {}).get("record", {}).get("loss", 0)
            # Bullet
            bullet_rating_best = stats.get("chess_bullet", {}).get("best", {}).get("rating", 0)
            bullet_rating_last = stats.get("chess_bullet", {}).get("last", {}).get("rating", 0)
            bullet_rating_rd = stats.get("chess_bullet", {}).get("last", {}).get("rd", 0)
            bullet_record_win = stats.get("chess_bullet", {}).get("record", {}).get("win", 0)
            bullet_record_draw = stats.get("chess_bullet", {}).get("record", {}).get("draw", 0)
            bullet_record_loss = stats.get("chess_bullet", {}).get("record", {}).get("loss", 0)
            return {
                "username": username,
                "fide": fide,
                "rapid_rating_best": rapid_rating_best,
                "rapid_rating_last": rapid_rating_last,
                "rapid_rating_rd": rapid_rating_rd,
                "rapid_record_win": rapid_record_win,
                "rapid_record_draw": rapid_record_draw,
                "rapid_record_loss": rapid_record_loss,
                "blitz_rating_best": blitz_rating_best,
                "blitz_rating_last": blitz_rating_last,
                "blitz_rating_rd": blitz_rating_rd,
                "blitz_record_win": blitz_record_win,
                "blitz_record_draw": blitz_record_draw,
                "blitz_record_loss": blitz_record_loss,
                "bullet_rating_best": bullet_rating_best,
                "bullet_rating_last": bullet_rating_last,
                "bullet_rating_rd": bullet_rating_rd,
                "bullet_record_win": bullet_record_win,
                "bullet_record_draw": bullet_record_draw,
                "bullet_record_loss": bullet_record_loss
            }
        else:
            print(username, response.status_code)
            return {
                "username": username
            }
    except Exception as error:
        print(username, error)
        return {
            "username": username
        }

### Get All Stats

In [130]:
title_stats = []

for chunk_title_players in tqdm(chunks_title_players):
    usernames = list(map(lambda title_player: title_player["username"], chunk_title_players))
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for username in usernames:
            futures.append(executor.submit(get_stats, username=username))
        for future in concurrent.futures.as_completed(futures):
            stats = future.result()
            title_stats.append(stats)
    title_stats_data_frame = pandas.DataFrame.from_dict(title_stats)
    title_stats_data_frame.to_csv(r'data/csv/title_stats.csv', index = False, header = True)

  0%|          | 0/558 [00:00<?, ?it/s]

esquiviaeusebius HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
 HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
estudiaajedrez HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
estacio99 HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
esskeettit HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
essequamvideris HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
et_050673g HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
ethanli-ny HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
estudiando_ajedrez HTTPSConnectionPool(host='homapis.vercel.app', port=443): Read timed out. (read timeout=10)
eugenedelacroix HTTPSConnectionPool(host='homapis.vercel.app'